In [ ]:
# B.Tech Project, Indian Insititute of Technology, Delhi
# Authors - Harshita (2019CS190272), Om Krishna (2019CS10272)
# Input : Lineament rasters fetched using GET requests from Bhuvan 


# !pip install rasterio
import requests
import rasterio as rio
from rasterio.enums import Resampling
from rasterio.plot import show
import numpy as np
from matplotlib import pyplot
from PIL import Image
from osgeo import gdal
import os.path
import time
import math

In [ ]:
import csv

# CSV file with state Lat1 Long1 Lat2 Long2, and state codes
# https://docs.google.com/spreadsheets/d/1Te7I-JH1sTqC_e6MOMegvVFpeUneAt7c7NYqn3A787Q/edit?usp=sharing
# ['AndamanNicobar', '92.208', '6.758', '93.948', '13.676', 'AN']

with open('lintemp/bhuvan_extent.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)

data = data[1:]

In [ ]:
folder = 'lin_newres/' # output folder
for s in data :

    # GET DATA FROM BHUVAN # 
    lineament_path = folder + str(s[0]) + '_lin.tif'
    
    URL = "https://bhuvan-vec2.nrsc.gov.in/bhuvan/wms"
    
    # Normalising resolution according to state height/width
    alpha = 2
    width = 256 * math.ceil(float(s[3]) - float(s[1])) * alpha
    height = 256 * math.ceil(float(s[4]) - float(s[2])) * alpha

    # Get request parameters
    PARAMS = {
        'LAYERS':'lineament:' + s[5] + '_LN50K_0506',
        'SERVICE':'WMS',
        'VERSION':'1.1.1',
        'REQUEST':'GetMap',
        'FORMAT':'image/png',
        'SRS':'EPSG:4326',
        'BBOX': str(s[1]) + ',' + str(s[2]) + ',' + str(s[3]) + ',' + str(s[4]),  # L1 L2 L3 L4
        'WIDTH': str(width),
        'HEIGHT': str(height)
    }

    resp = requests.get(url = URL, params = PARAMS)

    # create image (.tif)
    if resp.status_code == 200:
        with open(lineament_path, 'wb') as f:
            for chunk in resp:
                f.write(chunk)

    image = pyplot.imread(lineament_path, 0)
    pyplot.imshow(image)
    
    # CREATE MASK #
    lineament_mask = folder + str(s[0]) + '_lin_mask.tif'
    while not os.path.exists(lineament_path):
        time.sleep(1)
    raster = rio.open(lineament_path)

    r = raster.read(1).astype(float)
    g = raster.read(2).astype(float)
    b = raster.read(3).astype(float)

    output = np.empty(raster.shape)
    m,n = raster.shape
    for i in range(m):
      for j in range(n):
        if r[i][j]==255 and g[i][j]==255 and b[i][j]==255: # white = no lineament
          output[i][j] = 0
        else:
          output[i][j] = 1

    out_tiff = Image.fromarray(output)
    out_tiff.save(lineament_mask)
    
    # GEOREFERENCE MASK #
    georeffed_lineament_mask = folder + 'georeff/' + str(s[0]) + '_lin_georeff.tif'
    while not os.path.exists(lineament_mask):
        time.sleep(1)
    ds = gdal.Open(lineament_mask)

    options_list = [
        '-of GTiff',
        '-a_srs EPSG:4326',
        '-a_ullr ' + str(s[1]) + ' ' + str(s[4]) + ' ' + str(s[3]) + ' ' + str(s[2]) # L1 L4 L3 L2
    ] 
    options_string = " ".join(options_list)

    ds = gdal.Translate(georeffed_lineament_mask,
                        lineament_mask,
                        options=options_string)

    ds = None
    
    # PROXIMITY ANALYSIS 
    # Masking pixels which are within 2m of a lineament
    proximity_district_mask = folder + str(s[0]) + '_proximity_mask.tif'

    drv = gdal.GetDriverByName("GTiff")
    pds = drv.Create(proximity_district_mask, width, height, 1, gdal.GDT_Float32)

    while not os.path.exists(georeffed_lineament_mask):
        time.sleep(1)
    ds = gdal.Open(georeffed_lineament_mask)

    options_list = [
        'MAXDIST = 2',
        'DISTUNITS = GEO'
    ] 

    pds = gdal.ComputeProximity(ds.GetRasterBand(1), 
                               pds.GetRasterBand(1),
                               options=options_list)

    pds = None
    
    # 2M BUFFER MASK #
    buffer_mask = folder + str(s[0]) + '_buffer_mask.tif'
    while not os.path.exists(proximity_district_mask):
        time.sleep(1)
    raster = rio.open(proximity_district_mask)

    x = raster.read(1).astype(float)

    output = np.empty(raster.shape)
    m,n = raster.shape
    for i in range(m):
      for j in range(n):
        if x[i][j] <= 2 :   # 2m buffer
          output[i][j] = 1
        else:
          output[i][j] = 0

    out_tiff = Image.fromarray(output)
    out_tiff.save(buffer_mask)
    
    # FINAL PROXIGEO MASK #
    georeffed_proximity_mask = folder + 'proxigeo_masks/' + str(s[0]) + '_proxigeo_mask.tif'
    while not os.path.exists(buffer_mask):
        time.sleep(1)
    ds = gdal.Open(buffer_mask)

    options_list = [
        '-of GTiff',
        '-a_srs EPSG:4326',
        '-a_ullr ' + str(s[1]) + ' ' + str(s[4]) + ' ' + str(s[3]) + ' ' + str(s[2]) # L1 L4 L3 L2
    ] 
    options_string = " ".join(options_list)

    ds = gdal.Translate(georeffed_proximity_mask,
                        buffer_mask,
                        options=options_string)

    ds = None
    
    print(str(s[0]) + " Lineament Buffer Generation Complete.")
    

# ALL INDIA LINEAMENTS
# processing.run("gdal:merge", {'INPUT':[...],
#                               'PCT':False, 'SEPARATE':False,
#                               'NODATA_INPUT':0, 'NODATA_OUTPUT':None,
#                               'OPTIONS':'','EXTRA':'', 'DATA_TYPE':5,'OUTPUT':'TEMPORARY_OUTPUT'})